# 1. Prepare Data

In [1]:
import os
import tensorflow as tf

In [2]:
genres = ['Drama', 'Adventure', 'Action', 'Crime', 'Comedy', 'Thriller', 'Animation', 'Mystery', 'Romance', 'Biography']
budget_lst = [
        "under 1 million USD",
        "1-10 million USD",
        "10-25 million USD",
        "25-50 million USD",
        "50 million USD or more"
    ]
def split_label(path):
    filename = os.path.basename(path)
    label_part = filename.strip()[0:16]
    label = [int(bit) for bit in label_part if bit.isdigit()]
    # one hot

    return label

def decode_label(label_part):
    genre_bits = label_part[:10]  # first 10 bits for genres
    budget_bits = label_part[10:15]  # next 5 bits for budgets
    genres_label = []
    budget_label = ""
    for i in range(10):
        if genre_bits[i] == 1:
            genres_label.append(genres[i])
    for i in range(5):
        if budget_bits[i] == 1:
            budget_label = budget_lst[i]

    return {
        'genres': genres_label,
        'budget': budget_label
    }
example_path = "./database/1110000000 00100 10152.jpg"
example_label = split_label(example_path)
print(example_label)
print(decode_label(example_label))



[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
{'genres': ['Drama', 'Adventure', 'Action'], 'budget': '10-25 million USD'}


In [3]:
def load_and_preprocess_image(path, label):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [224, 224])  # Resize to the expected input size of your model
    image = image / 255.0  # Normalize the image to [0, 1] range
    return image, label

def create_dataset_from_directory(directory="database"):
    # Collect all paths and labels
    paths, labels = [], []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(".jpg"):  # Assuming JPEG images
                path = os.path.join(root, file)
                paths.append(path)
                label = split_label(path)
                labels.append(label)
    # Convert lists to tensor slices
    path_ds = tf.data.Dataset.from_tensor_slices(paths)
    label_ds = tf.data.Dataset.from_tensor_slices(labels)
    image_label_ds = tf.data.Dataset.zip((path_ds, label_ds))

    # Map the load and preprocess function
    image_label_ds = image_label_ds.map(load_and_preprocess_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    return image_label_ds

dataset = create_dataset_from_directory()
dataset = dataset.shuffle(1000).batch(32).prefetch(tf.data.experimental.AUTOTUNE)

# Example iteration over the dataset
for images, labels in dataset.take(1):  # Take 1 batch
    print(images.shape)  # Output the shape of images, e.g., (32, 224, 224, 3)
    print(labels)        # Output the corresponding labels

# Split to train valid test
train_size = int(0.6 * len(dataset))
valid_size = int(0.2 * len(dataset))
test_size = len(dataset) - train_size - valid_size
train_dataset = dataset.take(train_size)
valid_dataset = dataset.skip(train_size).take(valid_size)
test_dataset = dataset.skip(train_size + valid_size).take(test_size)
print(len(train_dataset), len(valid_dataset), len(test_dataset))


(32, 224, 224, 3)
tf.Tensor(
[[1 0 1 1 0 0 0 0 0 0 0 0 0 0 1]
 [1 1 0 0 1 0 0 0 0 0 1 0 0 0 0]
 [0 1 0 0 1 0 1 0 0 0 0 0 0 0 1]
 [1 1 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [1 1 1 0 0 0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 1 0 0 0 1 0 0 0 1 0 0]
 [1 1 1 0 0 0 0 0 0 0 0 0 0 0 1]
 [1 0 0 0 1 0 0 0 0 1 0 0 1 0 0]
 [0 1 1 0 0 0 0 0 0 0 0 1 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
 [1 1 1 0 0 0 0 0 0 0 0 0 0 0 1]
 [0 1 0 0 1 0 1 0 0 0 0 0 0 0 1]
 [0 0 0 0 1 0 0 0 1 0 0 0 1 0 0]
 [0 1 1 0 0 0 0 0 0 0 0 0 0 0 1]
 [1 0 0 1 0 0 0 1 0 0 0 1 0 0 0]
 [0 1 0 0 1 0 1 0 0 0 0 0 0 0 1]
 [1 1 1 0 0 0 0 0 0 0 0 0 1 0 0]
 [1 0 0 1 0 0 0 0 0 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 1 0 0 0 0 0 1 0]
 [1 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
 [0 1 1 0 0 0 0 0 0 0 0 0 0 1 0]
 [1 1 1 0 0 0 0 0 0 0 0 0 0 0 1]
 [0 1 1 0 0 0 0 0 0 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 0 0 1 0 0 1 0 0]
 [1 0 1 0 0 0 0 0 0 0 0 0 0 0 1]
 [1 1 0 0 0 1 0 0 0 0 0 1 0 0 0]
 [1 0 0 0 0 1 0 1 0 0 0 0 0 1 0]
 [0 0 0 1 1 0 0 0 0 0 0 1 0 0 0]
 [1 0 0 1 0 0 0 1 0 0 0 1 0 0 0]
 [1 0 0 0 0 0 

2024-04-19 08:16:42.484173: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


# 2. Define Model

In [4]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras import layers

In [5]:
# Load the VGG16 base model, pretrained on ImageNet
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze the convolutional base

# Add custom layers on top for your specific task
x = Flatten()(base_model.output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(15, activation='sigmoid')(x)  # Adjust the number of outputs and activation function based on your task

# Create the final model
model = Model(inputs=base_model.input, outputs=output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [6]:
x = layers.Flatten()(base_model.output)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(0.5)(x)
output = layers.Dense(15, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


# 3. Model Training

In [7]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

In [8]:
# Callback to save the best model
checkpoint = ModelCheckpoint('best_model.keras', save_best_only=True, monitor='val_loss', mode='min')

# Callback to reduce learning rate when a metric has stopped improving
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)

# Callback for early stopping
early_stopping = EarlyStopping(monitor='val_accuracy', patience=1, verbose=1, mode='max', restore_best_weights=True)

# Train the model with added early stopping
history = model.fit(
    train_dataset,
    epochs=20,
    validation_data=valid_dataset,
    callbacks=[checkpoint, reduce_lr, early_stopping]
)

Epoch 1/20
277/277 ━━━━━━━━━━━━━━━━━━━━ 1803s 7s/step - accuracy: 0.5634 - loss: 0.5690 - val_accuracy: 0.6916 - val_loss: 0.4236 - learning_rate: 0.0010
Epoch 2/20
277/277 ━━━━━━━━━━━━━━━━━━━━ 1936s 7s/step - accuracy: 0.6720 - loss: 0.4431 - val_accuracy: 0.6940 - val_loss: 0.4117 - learning_rate: 0.0010
Epoch 3/20
277/277 ━━━━━━━━━━━━━━━━━━━━ 1854s 7s/step - accuracy: 0.6669 - loss: 0.4232 - val_accuracy: 0.6878 - val_loss: 0.4071 - learning_rate: 0.0010
Epoch 3: early stopping
Restoring model weights from the end of the best epoch: 2.


# 4. Evaluation and Iteration

In [9]:
test_loss, test_acc = model.evaluate(test_dataset)  # Assuming 'test_dataset' is prepared
print("Test accuracy:", test_acc)

94/94 ━━━━━━━━━━━━━━━━━━━━ 479s 5s/step - accuracy: 0.6812 - loss: 0.4325
Test accuracy: 0.6800804734230042
